In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib as mp

# Import csv

## Main import

In [2]:
with open('../data/ESP_PUBLIC.IDENTITE_ARBRE.csv', newline='', encoding='utf-8-sig') as csvfile:
    # use "," separator to generate column value as list element
    # cs.reader return a class object of "_csv.reader"
    dataTemp = csv.reader(csvfile, delimiter=',')
    
    # Use list() to convert the csv object into a list, of list, depending on how the object is built
    # Hard to understand how list(csv_object) behaves 
    data = list(dataTemp)

## Copy the dataset to advoid re-import

In [3]:
dataset = data

## Utils

In [4]:
def get_header(data:list) -> list:
    if len(data) > 0:
        return data[0]
    else:
        raise ValueError("The dataset sent is empty...")

In [5]:
def get_dict_colName_colNum(data:list) -> dict:
    dictTemp = {}
    
    for el in get_header(data):
        dictTemp[el] = get_header(data).index(el)
        
    return dictTemp
# print(get_dict_colName_colNum(dataset))

In [6]:
def is_in_header(colName:str, data:list) -> bool:
    return colName in get_header(data)
# print(is_in_header("GENRE_DESC", dataset))

In [45]:
def average(col:list) -> float:
    for el in col:
        if type(el) is not float:
            raise TypeError("The list sent doesn't have float or contains empty string")
    return sum(col)/len(col)

# Iteration 1 : Main computation

## Afficher deux premieres lignes

In [7]:
# Slice and extract the first two list ie the first two lignes
header = get_header(dataset)
lignes2 = dataset[0:2]

specDataset = {
    "rows" : len(dataset),
    "columns" : len(dataset[0])
}

## Afficher colonne n°2

In [8]:
# We see that to access the row is [i] after lignes2. 
# Then we have the list corresponding to the row i-1 (start from 0)
# In this list, we use [i] to acces the i element corresponding to the column
ligne1_col2 = lignes2[0][1]
ligne2_col2 = lignes2[1][1]

In [9]:
print(ligne1_col2 + " : " + ligne2_col2)

CODE : ESP29285


## Get 50 first & last

In [10]:
# Should be called when having a list coming from : get_col_values()
def extract_50(col:list) -> list:
    if len(col) > 100:
        return col[:50]+ ["------> more values"] + col[len(col)-50:]
    else:
        raise ValueError(f"List {col} has less than 100 elements")
# extract_50(colList)

## Infos année plantation data

In [11]:
print(get_header(dataset))

['ELEM_POINT_ID', 'CODE', 'NOM', 'GENRE', 'GENRE_DESC', 'CATEGORIE', 'CATEGORIE_DESC', 'SOUS_CATEGORIE', 'SOUS_CATEGORIE_DESC', 'CODE_PARENT', 'CODE_PARENT_DESC', 'ADR_SECTEUR', 'BIEN_REFERENCE', 'GENRE_BOTA', 'ESPECE', 'VARIETE', 'STADEDEDEVELOPPEMENT', 'EQUIPE', 'REMARQUES', 'ANNEEDEPLANTATION', 'RAISONDEPLANTATION', 'TRAITEMENTCHENILLES', 'COURRIER', 'IDENTIFIANTPLU', 'TYPEIMPLANTATIONPLU', 'INTITULEPROTECTIONPLU', 'ANNEEABATTAGE', 'ESSOUCHEMENT', 'DIAMETREARBRE', 'CAUSEABATTAGE', 'COLLECTIVITE', 'GeoJSON']


In [12]:
def get_col_value_from_name(colName:str, data:list) -> str:
    
    if is_in_header(colName, data):
        return get_dict_colName_colNum(data)[colName]
    
# print(get_col_value_from_name(colName="CODE_PARENT", data=dataset))

In [13]:
def get_col_values(colName:str, data:list) -> list:
    
    col = [el[get_col_value_from_name(colName, data)] for el in data]
    
    return col
# print(get_col_values("ANNEEDEPLANTATION", dataset))

#### Trees column parsing utils

In [14]:
colToStudy = "ANNEEDEPLANTATION"

In [15]:
colList = get_col_values(colToStudy, dataset)
print("there is --> " + str(len(colList)) + " <-- trees studied")

there is --> 31620 <-- trees studied


In [16]:
def count_empty_value(col:list) -> int:
    
    emptyList = [el for el in col if not el]
    
    return len(emptyList)

In [17]:
print("there is --> " + str(count_empty_value(colList)) + " <-- empty values in " + colToStudy )

there is --> 1439 <-- empty values in ANNEEDEPLANTATION


In [35]:
def remove_empty(col:list) -> list:
    
    return [el for el in col if el]

In [61]:
def get_min(col:list) -> int:
    
    colValues = remove_empty(col)
    
    return min(colValues[])

In [62]:
def get_max(col:list) -> int:
    
    colValues = remove_empty(col)
    
    return max(colValues)

#### Count trees for each year in dataset

In [63]:
def count_specific_value_in_col(value:str, col:list) -> int:
    
    valuesCorresponding = [el for el in col if str(el) == str(value)]  
    
    return len(valuesCorresponding)

In [64]:
valueToFind = "1994"
print(f"There was {count_specific_value_in_col(valueToFind, colList)} trees planted in {valueToFind}")

There was 252 trees planted in 1994


In [65]:
print(f"The oldest year of plantation is : {get_min(colList)}")
print(f"The last year of plantation is : {get_max(colList)}")

TypeError: unsupported operand type(s) for -=: 'list' and 'str'

#### Count trees for each year in dataset

In [49]:
# This method compute for each year the number of trees planted
# Then it compute the average taking into account only the trees where the year is known
def average_trees_planted(col:list) -> dict:
    cleanList = remove_empty(col)
    removeDuplicate = set(cleanList)
    
    nbTreesPerYear = [float(count_specific_value_in_col(el,col)) for el in removeDuplicate]
    
    return average(nbTreesPerYear)
        
print(round(average_trees_planted(colList),2))

457.29


In [ ]:
# This function compute the average taking into account
# The total number of trees in the dataset
# The max and min year known

def rought_average(col:list) -> float:
    